<a href="https://colab.research.google.com/github/Syed-Raza-Ali/Browsing_AI_Agent/blob/main/Browsing_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Tavily-Enhanced Real-Time Browsing Agent**

- **A dynamic browsing agent leveraging LangGraph and the Tavily real-time search API to provide instant, accurate responses to user queries. Powered by the Gemini-1.5-Flash LLM and integrated tools, the agent retrieves, processes, and delivers relevant information in a user-friendly format. Designed for seamless and efficient communication, this agent ensures users get the insights they need without technical jargon or raw data, making it ideal for real-time information access.**

In [15]:
# install the required packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [16]:
# API KEYS VARIABLES
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
# GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")


In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults


# Initialize Tavily search tool
tavily_search = TavilySearchResults(max_results = 3)

# Maintain chat history
chat_history = []

def stream_graph_updates(user_input):
    """A placeholder function for processing graph updates."""
    print(f"Processing graph updates for: {user_input}")


print("Hello there! I am an browsing agent. Please give me a prompt, and I will bring you the answer by searching on Google")
print("Type 'exit' , 'quit' , 'q' , 'bye' for ending")


def main():
  """ Get realtime information using tavily search """

  while True:


    try:
        # Prompt the user for input
        print("\n")
        user_input = input("User : ").strip()

        # Exit condition
        if user_input.lower() in ["quit", "exit", "q","bye"]:
            print("Bye! Have a great day!")
            break

        # Handle empty input
        if not user_input:
            print("No prompt found. Please type something.")
            continue

        # Add user input to chat history
        chat_history.append({"role": "user", "content": user_input})

        # Call the placeholder function (replace as needed)
        try:
            stream_graph_updates(user_input)
        except NameError:
            # Handle missing function gracefully
            print("stream_graph_updates is not implemented. Skipping.")

        # Perform a Tavily search if the function is not implemented
        search_docs = tavily_search.invoke(user_input)

        # Add Tavily response to chat history
        chat_history.append({"role": "assistant", "content": str(search_docs)})

        # Display results from Tavily search
        print("Search Results:")
        for i, doc in enumerate(search_docs, start=1):
            print(f"\nResult {i}:")
            print(f"Here is the {i} documentaion from web search you can get your answer from here:")
            print(f"Documentation: {doc.get('content', 'N/A')}")
            print(f"URL: {doc.get('url', 'N/A')}")

    except Exception as e:
        print(f"An error occurred: {e}")
        break

    print("\nChat History:")
    for message in chat_history:
      role = message["role"]
      content = message["content"]
      print(f"{role.capitalize()}: {content}")



main()

Hello there! I am an browsing agent. Please give me a prompt, and I will bring you the answer by searching on Google
Type 'exit' , 'quit' , 'q' , 'bye' for ending


User : who is the prime minister of Pak in 2024
Processing graph updates for: who is the prime minister of Pak in 2024
Search Results:

Result 1:
Here is the 1 documentaion from web search you can get your answer from here:
Documentation: In this photo released by the Pakistan’s President Office, President Arif Alvi, right, administers the oath of office to newly elected Prime Minister Shehbaz Sharif during a ceremony at the Presidential Palace, in Islamabad, Pakistan, Monday, March 4, 2024. In this photo released by the Pakistan’s President Office, President Arif Alvi, right, administers the oath of office to newly elected Prime Minister Shehbaz Sharif during a ceremony at the Presidential Palace, in Islamabad, Pakistan, Monday, March 4, 2024. In this photo released by the Pakistan’s President Office, President Arif Alvi, 